<a href="https://colab.research.google.com/github/niteshgupta2711/hyper_personized_reinforced_recommendation_system/blob/master/prototype1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:





dataset=pd.read_csv('../data/AMAZON_FASHION.csv',names=['index','pID','image','ratings','reviewtext','time','cust_id','cust_name','style','summary','unixtime','verified','vote'],low_memory=False)

In [ ]:
dataset.head()
#prod_emb_model_final
#final_logs

,index,pID,image,ratings,reviewtext,time,cust_id,cust_name,style,summary,unixtime,verified,vote
0,NaN,asin,image,overall,reviewText,reviewTime,reviewerID,reviewerName,style,summary,unixReviewTime,verified,vote
1,0.0,7106116521,NaN,5,Exactly what I needed.,"10 20, 2014",A1D4G1SNUZWQOT,Tracy,NaN,perfect replacements!!,1413763200,True,NaN
2,1.0,7106116521,NaN,2,"I agree with the other review, the opening is ...","09 28, 2014",A3DDWDH9PX2YX2,Sonja Lau,NaN,"I agree with the other review, the opening is ...",1411862400,True,3.0
3,2.0,7106116521,NaN,4,Love these... I am going to order another pack...,"08 25, 2014",A2MWC41EW7XL15,Kathleen,NaN,My New 'Friends' !!,1408924800,False,NaN
4,3.0,7106116521,NaN,2,too tiny an opening,"08 24, 2014",A2UH2QQ275NV45,Jodi Stoner,NaN,Two Stars,1408838400,True,NaN


In [ ]:
# building the environment with data 
# this will give positive reinforcements for the agent to learn
# using Actor crtic approach
# temporal difference
# q learning
# sarsa
# deep sarsa
# actor2Crtic

dataset.ratings.unique()


array(['overall', '5', '2', '4', '3', '1'], dtype=object)

In [ ]:
len(dataset.cust_id.unique())

749234

In [ ]:
len(dataset)

883637

In [ ]:
len(dataset.pID.unique())

186190

In [ ]:
# lets understand the data first


In [ ]:
dataset=dataset.drop(['index','image','time','cust_name','style','reviewtext','unixtime'],axis=1)[1:]

In [ ]:
dataset[dataset.ratings=='overall']

,pID,ratings,cust_id,summary,verified,vote


In [ ]:
dataset.vote.isnull().sum()

803736

In [ ]:
803736/883637

0.9095771227325248

In [ ]:
# 10 % data in vote is not present
dataset.info()
import numpy as np

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883636 entries, 1 to 883636
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   pID       883636 non-null  object
 1   ratings   883636 non-null  object
 2   cust_id   883636 non-null  object
 3   summary   883099 non-null  object
 4   verified  883636 non-null  object
 5   vote      79900 non-null   object
dtypes: object(6)
memory usage: 40.4+ MB


In [ ]:
# we are going to embed the pID(product ids) baed on ratings 
# getting the dataset for embedding the profuct id
# we will be embedding the product id s in latent space 128 dimensions

In [ ]:
embedding_dataset=dataset.drop(['cust_id','verified','vote'],axis=1)

In [ ]:
# embedding_dataset.head()
# l=[]
# embedding_dataset['summary'].apply(lambda review:[l.append(x) for x in review.split()])

In [ ]:
Xtrain,ytrain=embedding_dataset.drop(['ratings'],axis=1),embedding_dataset['ratings'].apply(lambda x:str(int(x)-1)).astype(np.int64)

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [ ]:
Xtrain.summary=Xtrain.summary.fillna('')

In [ ]:
product_embeds=TextVectorization(max_tokens=186191,output_sequence_length=1)

In [ ]:
# summary_embeds=TextVectorization(max_tokens=6000,output_sequence_length=30)
product_embeds.adapt(Xtrain.pID.unique())

In [ ]:
# summary_embeds.adapt(Xtrain.summary)

In [ ]:
product_embeds(Xtrain.tail()['pID'].values)

<tf.Tensor: shape=(5, 1), dtype=int64, numpy=
array([[2],
       [2],
       [2],
       [3],
       [9]], dtype=int64)>

In [ ]:
Xtrain.pID.unique()

array(['7106116521', 'B00007GDFV', 'B00008JOQI', ..., 'B01HJHTH5U',
       'B01HJHF97K', 'B01HJG5NMW'], dtype=object)

In [ ]:
# product embeds completed

In [ ]:
product_input=tf.keras.Input(shape=(),dtype=tf.string)
te=product_embeds(product_input)
embedding=tf.keras.layers.Embedding(input_dim=186191,output_dim=128, input_length=1,)(te)
sq=tf.keras.backend.squeeze(embedding,1)
model1=tf.keras.Model(product_input,sq)

In [ ]:
# model1(np.array(Xtrain.pID.values[10:12]))

In [ ]:
import tensorflow_hub as hub

embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",trainable=False,input_shape=[])
embeddings = embed(["product was n"])

# this willl take more time to load as it is 1 gb module loaded
#print(embeddings)
# embL=hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",trainable=False,input_shape=[])

# The following are example embedding output of 512 dimensions per sentence
# Embedding for: The quick brown fox jumps over the lazy dog.
# [-0.03133016 -0.06338634 -0.01607501, ...]
# Embedding for: I am a sentence for which I would like to get its embedding.
# [0.05080863 -0.0165243   0.01573782, ...]


In [ ]:
summary_input=tf.keras.Input(shape=(),dtype=tf.string)
use=embed(summary_input)
final=tf.keras.layers.Dense(128,activation='relu')(use)
model2=tf.keras.Model(summary_input,final)



In [ ]:
# model2(Xtrain.summary.values[1:4])

In [ ]:
# model1.outputs


In [ ]:
x=tf.keras.layers.Concatenate()([model1.output,model2.output])

In [ ]:
dense_1= tf.keras.layers.Dense(128,activation='relu')(x)
dense_2=tf.keras.layers.Dense(5, activation='softmax')(dense_1) 


In [ ]:
final_model=tf.keras.Model(inputs=[product_input,summary_input],outputs=dense_2)

In [ ]:
# img_file = './model_arch.png'

# tf.keras.utils.plot_model(final_model, to_file=img_file, show_shapes=True, show_layer_names=True)

In [ ]:
from tensorflow.keras.utils import plot_model


plot_model(final_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [ ]:
# !pip install netron
import os
os.getcwd()


'D:\\ALL_Python\\AI_ops\\PROJECTS\\hyper_personized_reinforced_recommendation_system\\notebooks'

In [ ]:
model1=tf.keras.models.load_model('prod_emb_model_final/prod_emb_model_final')

In [ ]:
# final_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 1)           0           ['input_1[0][0]']                
 ization)                                                                                         
                                                                                                  
 input_2 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 128)       23832448    ['text_vectorization[0][0]'

In [ ]:
data1=tf.data.Dataset.from_tensor_slices(Xtrain.pID.values)
data2=tf.data.Dataset.from_tensor_slices(Xtrain.summary.values)
target=tf.data.Dataset.from_tensor_slices(ytrain.values)
daata=tf.data.Dataset.zip(((data1,data2),target)).batch(32).prefetch(1)
# tf.data.Dataset.zip((features,labels))

In [ ]:
final_model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),)

In [ ]:
import netron

In [ ]:
final_model.save('model.h5')

NotImplementedError: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Received a model or layer TextVectorization with weights [<keras.layers.preprocessing.index_lookup.VocabWeightHandler object at 0x000001EF095FACD0>]

In [ ]:
# # final_model.fit(daata,epochs=1)
# model2(Xtrain.summary.values)

In [ ]:
netron.start('model',)

In [ ]:
import tensorboard
#!tensorboard --logdir='logs'
# launch the above in a shell 
# and look for metrics you want to see
# tensorboard dev upload --logdir notebooks/logs
# look at the realtime progress in tensorboard using above command


In [ ]:
# final_model.fit(daata,epochs=1,callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs')])


In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
%tensorboard --logdir notebooks/logs

Reusing TensorBoard on port 6006 (pid 4576), started 1 day, 10:20:25 ago. (Use '!kill 4576' to kill it.)

In [ ]:
# given a cust_id and producti_id give the environment rewards and s states
# when you get a product_id as  a recommendation , you need to get the embedding of that product_id as next state
# and again the cust_id and product_id we will get the env rewrds and states,
# we will end the recommendation when it recommends ""
# this is the current plan
#  we will use A2C technique in reinforcement learining to reforce recommendation 

In [ ]:
# we will be using sklearn minibatchKmeans algorihtm for clustering algorithm

In [ ]:
# once you get the cust_id get all the products bought by that customer, and the product_id that got recommended
# check whether or not recommended product is similar to products bought by the customer
# get the recommended product_ids embeds
# get the products bought by the customer embeds

In [ ]:
product_embeds.get_vocabulary()[7]

'b01hjgx04a'

In [ ]:
product_embeds('b000yfsr5g')

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([182638], dtype=int64)>

In [ ]:
# model1(np.array([['b000yfsr5g',]]))

In [ ]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np


In [ ]:
# class recommendation_Env:
    
#     def __init__(self,data_path,text_vectorizer,emb_model,recusive_model):
#         self.dataset=pd.read_csv(data_path,names=['index','pID','image','ratings','reviewtext','time','cust_id','cust_name','style','summary','unixtime','verified','vote'],low_memory=False)
#         self.t=text_vectorizer
#         self.vocab=self.t.get_vocabulary()
#         self.pr_model=emb_model
#         self.gcwd=self._get_cluster_with_data()
#         self.r_model=recursive_model
    
#     def _get_products(self,cust_id:str):
#         return self.dataset[self.dataset.cust_id==cust_id]['pID'].values
    
#     def _get_cluster_with_data(self):
#         self.kmeans = MiniBatchKMeans(n_clusters=5,random_state=42,batch_size=1024,max_iter=100)
#         self.kmeans.fit(self.pr_model(np.array(self.vocab).reshape(-1,1)).numpy())
#         return self.kmeans
    
#     def _get_pr_from_index(self,index):
#         return self.vocab[index]
    
#     def reset(self):
#         return '[UNK]'
    
#     def _get_cluster_index(self,product_embedding):
#         #self.cluster=self._embeds_with_data()
        
#         return self.gcwd.predict(product_embedding)
    
#     def _get_emb_product(self,pr_id):
#         return self.pr_model(np.array([[self._get_pr_from_index(pr_id)]]))
    
#     def _get_cluster_from_cust_id(self,index_for_recommended_product,cust_id):
#         self.embeds_from_cust_id=self.pr_model(self._get_products(cust_id)).numpy()
#         self.cluster_ids_for_customer=self._get_cluster_index(self.embeds_from_cust_id)
#         self.embeds_from_recommenation=self.pr_model(np.array([[self._get_pr_from_index(index_for_recommended_product)]])).numpy()
#         self.cluster_id_for_recomendation=self._get_cluster_index(self.embeds_from_recommenation)
#         return self.cluster_ids_for_customer,self.cluster_id_for_recomendation
    
#     def step(self,action,agent):
#         emb__=self._get_emb_product(action)
#         next_state=
        
        
        
        
        
    
    
        
        
        
    
    
    
    
        
        

In [ ]:
# env=recommendation_Env('../data/AMAZON_FASHION.csv',product_embeds,model1)

In [ ]:
# env._get_products('A1D4G1SNUZWQOT')
# kp=model1(np.array([['A1D4G1SNUZWQOT']])).numpy()

In [ ]:
# # model1(np.array(product_embeds.get_vocabulary()).reshape(-1,1)).numpy()
# env._get_cluster_index(kp)


In [ ]:
# env._get_cluster_from_cust_id('A3O90PACS7B61K')

In [ ]:
dataset.cust_id.values

array(['A1D4G1SNUZWQOT', 'A3DDWDH9PX2YX2', 'A2MWC41EW7XL15', ...,
       'A3O90PACS7B61K', 'A2HO94I89U3LNH', 'A2RSX9E79DUHRX'], dtype=object)

In [ ]:
# env._get_emb_product(456)

In [ ]:
model2.save('model.h5')

In [ ]:
netron.start('model.h5')

Serving 'model.h5' at http://localhost:8080


('localhost', 8080)

In [ ]:
# kmeans = MiniBatchKMeans(n_clusters=5,
#                          random_state=42,
#                           batch_size=1024,
#                         max_iter=100)

In [ ]:
# kmeans.fit(model1(np.array(product_embeds.get_vocabulary()).reshape(-1,1)).numpy())

In [ ]:
# kl=kmeans.predict(model1(np.array(product_embeds.get_vocabulary()).reshape(-1,1)).numpy())

In [ ]:
# set(kl)

In [ ]:





dataset=pd.read_csv('../data/AMAZON_FASHION.csv',names=['index','pID','image','ratings','reviewtext','time','cust_id','cust_name','style','summary','unixtime','verified','vote'],low_memory=False)

In [ ]:
dataset.head()

,index,pID,image,ratings,reviewtext,time,cust_id,cust_name,style,summary,unixtime,verified,vote
0,NaN,asin,image,overall,reviewText,reviewTime,reviewerID,reviewerName,style,summary,unixReviewTime,verified,vote
1,0.0,7106116521,NaN,5,Exactly what I needed.,"10 20, 2014",A1D4G1SNUZWQOT,Tracy,NaN,perfect replacements!!,1413763200,True,NaN
2,1.0,7106116521,NaN,2,"I agree with the other review, the opening is ...","09 28, 2014",A3DDWDH9PX2YX2,Sonja Lau,NaN,"I agree with the other review, the opening is ...",1411862400,True,3.0
3,2.0,7106116521,NaN,4,Love these... I am going to order another pack...,"08 25, 2014",A2MWC41EW7XL15,Kathleen,NaN,My New 'Friends' !!,1408924800,False,NaN
4,3.0,7106116521,NaN,2,too tiny an opening,"08 24, 2014",A2UH2QQ275NV45,Jodi Stoner,NaN,Two Stars,1408838400,True,NaN


In [ ]:
dataset[dataset['cust_id']=='A1D4G1SNUZWQOT']

,index,pID,image,ratings,reviewtext,time,cust_id,cust_name,style,summary,unixtime,verified,vote
1,0.0,7106116521,NaN,5,Exactly what I needed.,"10 20, 2014",A1D4G1SNUZWQOT,Tracy,NaN,perfect replacements!!,1413763200,True,NaN


In [ ]:
#dataset.cust_id.unique().tolist()

In [ ]:
len(dataset)

883637

In [ ]:
749234/883637

0.8478979490446868

In [ ]:
cust_embeddings=TextVectorization(max_tokens=749235,output_sequence_length=1)

In [ ]:
cust_embeddings.adapt(dataset.cust_id.unique())

In [ ]:
cust_embeddings(dataset.cust_id.values[:10])

<tf.Tensor: shape=(10, 1), dtype=int64, numpy=
array([[     2],
       [676759],
       [279687],
       [425609],
       [383937],
       [152219],
       [499099],
       [155075],
       [686802],
       [ 99186]], dtype=int64)>

In [ ]:
# import pickle

In [ ]:
# with open('cust_emb.pkl','wb') as f:
#     pickle.dump(cust_embeddings.get_vocabulary(),f)

In [ ]:
len(cust_embeddings.get_vocabulary())

749235

In [ ]:
cust_inputd=tf.keras.Input(shape=(),dtype=tf.string)
cust_int=cust_embeddings(cust_inputd)
emb_c=tf.keras.layers.Embedding(input_dim=749235,output_dim=128)(cust_int)


In [ ]:
#final_inps=tf.keras.Input(shape=(128,))
#reshaped=tf.keras.layers.Reshape((1,128))(emb_c)
dense_1=tf.keras.layers.Dense(128,activation='relu',name='initial_dense_layer',)(emb_c)
norm_1=tf.keras.layers.BatchNormalization()(dense_1)
dense_2=tf.keras.layers.Dense(512,activation='relu')(norm_1)
actor=tf.keras.layers.Dense(186191,activation=None)(dense_2)
critic=tf.keras.layers.Dense(1)(dense_2)

In [ ]:
a2c_model=tf.keras.Model(cust_inputd,[critic,actor])

In [ ]:
a2c_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 text_vectorization_1 (TextVect  (None, 1)           0           ['input_1[0][0]']                
 orization)                                                                                       
                                                                                                  
 embedding (Embedding)          (None, 1, 128)       95902080    ['text_vectorization_1[0][0]']   
                                                                                                  
 initial_dense_layer (Dense)    (None, 1, 128)       16512       ['embedding[0][0]']          

In [ ]:
# final_model=tf.keras.Model(cust_inputd,[critic,actor])



In [ ]:
# netron.start('actor_model.h5')
prod_model=tf.keras.Model(dense_1,[critic,actor])
# this is recursiv model

In [ ]:
# dont use

In [ ]:
prod_model(model1(np.array([[['7106116521']]])))

[<tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[-0.18694483]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1, 186191), dtype=float32, numpy=
 array([[[ 2.5274879e-03, -6.9326963e-03,  9.4242732e-04, ...,
          -8.1937644e-05,  1.8577999e-03, -3.6084333e-03]]], dtype=float32)>]

In [ ]:
cust_emb_model=tf.keras.Model(cust_inputd,emb_c)

In [ ]:
a2c_model.save('a2c_model.h5')

In [ ]:
# netron.start('a2c_model.h5')
import tensorflow_probability as tfp

In [ ]:
class Agent:
    def __init__(self, r_model,emb_model,gamma=0.99):
        self.gamma = gamma
        # here embedding model is product embedding model
        self.opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
        self.actor_critic = a2c_model
        self.r_model=r_model
        self.pr_model=emb_model
        
    def get_action(self, state):
        
        _, action_probabilities = self.actor_critic(np.array([state]))
        action_probabilities = tf.nn.softmax(action_probabilities)
        action_probabilities = action_probabilities.numpy()
        dist = tfp.distributions.Categorical(probs=action_probabilities, dtype=tf.float32)
        action = dist.sample()
        return int(action.numpy()[0])
    
    def get_action_r(self,state):
        # the state here should be of shape (1,128)
        _, action_probabilities = self.r_model(np.array([state]))
        action_probabilities = tf.nn.softmax(action_probabilities)
        action_probabilities = action_probabilities.numpy()
        dist = tfp.distributions.Categorical(probs=action_probabilities, dtype=tf.float32)
        action = dist.sample()
        return int(action.numpy()[0])
    
    def actor_loss(self, prob, action, td):
        
        
        prob = tf.nn.softmax(prob)
        dist = tfp.distributions.Categorical(probs=prob,dtype=tf.float32)
        log_prob = dist.log_prob(action)
        loss = -log_prob * td
        return loss
        
    def learn(self, state, action, reward, next_state, done):
        state = np.array([state])
        next_state = np.array([next_state])
        with tf.GradientTape() as tape:
            value, action_probabilities = self.r_model(self.pr_model(np.array([[state]])), training=True)
            value_next_st, _ = self.r_model(self.pr_model(np.array([[next_state]])), training=True)
            td = reward + self.gamma * value_next_st * (1 - int(done)) - value
            actor_loss = self.actor_loss(action_probabilities, action, td)
            critic_loss = td ** 2
            total_loss = actor_loss + critic_loss
        grads = tape.gradient(total_loss, self.r_model.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.r_model.trainable_variables))
        return total_loss


In [ ]:
def train(agent, env, episodes,emb_model):
    # emb_model here is product emb model

    for episode in range(episodes):
        done = False
        state = '[UNK]'
        total_reward = 0
        all_loss = []
        while not done:
            action = agent.get_action_r(emb_model(np.array([state])))
            # print('___________got the action______________')
            next_state, reward, done = env.step(action,agent,state)
            print(next_state)
            loss = agent.learn(state, action, reward, next_state, done)
            all_loss.append(loss)
            state = next_state
            total_reward += reward
            if done:
                print("\n")
                print(f"Episode#:{episode} ep_reward:{total_reward}", end="\r")


In [ ]:
a2c_model(np.array([['[UNK]']]))

[<tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[0.0004417]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1, 186191), dtype=float32, numpy=
 array([[[ 9.4883656e-04,  1.8872769e-04,  6.9058716e-04, ...,
           2.1902465e-04,  9.9052489e-04, -4.5849956e-05]]], dtype=float32)>]

In [ ]:
agent=Agent(prod_model,model1)

In [ ]:
agent.get_action('[UNK]')

54481

In [ ]:
# class recommendation_Env:
    
#     def __init__(self,data_path,text_vectorizer,emb_model,recursive_model):
#         #emb_model is product embeds model
#         # text vecorizer is products text_vectorizer
#         # emb_model is model for product embedding model
#         self.dataset=pd.read_csv(data_path,names=['index','pID','image','ratings','reviewtext','time','cust_id','cust_name','style','summary','unixtime','verified','vote'],low_memory=False)
#         self.t=text_vectorizer
#         self.vocab=self.t.get_vocabulary()
#         self.pr_model=emb_model
#         self.gcwd=self._get_cluster_with_data()
#         self.r_model=recursive_model
    
#     def _get_products(self,cust_id:str):
#         return self.dataset[self.dataset.cust_id==cust_id]['pID'].values
    
#     def _get_cluster_with_data(self):
#         self.kmeans = MiniBatchKMeans(n_clusters=5,random_state=42,batch_size=1024,max_iter=100)
#         self.kmeans.fit(self.pr_model(np.array(self.vocab).reshape(-1,1)).numpy())
#         return self.kmeans
    
#     def _get_pr_from_index(self,index):
#         return self.vocab[index]
    
#     def reset(self):
#         return '[UNK]'
    
#     def _get_cluster_index(self,product_embedding):
#         #self.cluster=self._embeds_with_data()
        
#         return self.gcwd.predict(product_embedding)
    
#     def _get_emb_product(self,pr_id):
#         return self.pr_model(np.array([[self._get_pr_from_index(pr_id)]]))
    
#     def _get_cluster_from_cust_id(self,index_for_recommended_product,state):
#         #self.embeds_from_cust_id=self.pr_model(self._get_products(cust_id)).numpy()
#         self.embeds_from_cust_id=self.pr_model(np.array([[state]])).numpy()
#         self.cluster_ids_for_customer=self._get_cluster_index(self.embeds_from_cust_id)
#         self.embeds_from_recommenation=self.pr_model(np.array([[index_for_recommended_product]])).numpy()
#         self.cluster_id_for_recomendation=self._get_cluster_index(self.embeds_from_recommenation)
#         return self.cluster_id_for_recomendation.tolist(),self.cluster_ids_for_customer.tolist()
    
#     def step(self,action,agent,state):
# #         emb__1=self._get_emb_product(action)
# #         emb__1=emb__1[action]
#         #cr_,next_state=self.r_model(np.array([self._get_emb_product(action)]))
#         done=False
#         next_state_int=agent.get_action_r(self._get_emb_product(action))
        
#         if self._get_pr_from_index(next_state_int) in ('','[UNK]'):
#             done=True
#         next_state=self._get_pr_from_index(next_state_int)
#         customer_interests,recommendation=self._get_cluster_from_cust_id(next_state,state)
#         if recommendation[0] in customer_interests:
#             reward=2
#         reward=-1
#         return next_state,reward,done
        
        
    
        
        
        
    
    
        
        
        
    
    
    
    
        
        

In [ ]:
class recommendation_Env:
    
    def __init__(self,data_path,text_vectorizer,emb_model,recursive_model):
        #emb_model is product embeds model
        # text vecorizer is products text_vectorizer
        # emb_model is model for product embedding model
        self.dataset=pd.read_csv(data_path,names=['index','pID','image','ratings','reviewtext','time','cust_id','cust_name','style','summary','unixtime','verified','vote'],low_memory=False)
        self.t=text_vectorizer
        self.vocab=self.t.get_vocabulary()
        self.pr_model=emb_model
        self.gcwd=self._get_cluster_with_data()
        self.r_model=recursive_model
    
    def _get_products(self,cust_id:str):
        return self.dataset[self.dataset.cust_id==cust_id]['pID'].values
    
    def _get_cluster_with_data(self):
        self.kmeans = MiniBatchKMeans(n_clusters=5,random_state=42,batch_size=1024,max_iter=100)
        self.kmeans.fit(self.pr_model(np.array(self.vocab).reshape(-1,1)).numpy())
        return self.kmeans
    
    def _get_pr_from_index(self,index):
        return self.vocab[index]
    
    def reset(self):
        return '[UNK]'
    
    def _get_cluster_index(self,product_embedding):
        #self.cluster=self._embeds_with_data()
        
        return self.gcwd.predict(product_embedding)
    
    def _get_emb_product(self,pr_id):
        return self.pr_model(np.array([[self._get_pr_from_index(pr_id)]]))
    
    def _get_cluster_from_cust_id(self,index_for_recommended_product,state):
        #self.embeds_from_cust_id=self.pr_model(self._get_products(cust_id)).numpy()
        self.embeds_from_cust_id=self.pr_model(np.array([[state]])).numpy()
        self.cluster_ids_for_customer=self._get_cluster_index(self.embeds_from_cust_id)
        self.embeds_from_recommenation=self.pr_model(np.array([[index_for_recommended_product]])).numpy()
        self.cluster_id_for_recomendation=self._get_cluster_index(self.embeds_from_recommenation)
        return self.cluster_id_for_recomendation.tolist(),self.cluster_ids_for_customer.tolist()
    
    def step(self,action,agent,state):
#         emb__1=self._get_emb_product(action)
#         emb__1=emb__1[action]
        #cr_,next_state=self.r_model(np.array([self._get_emb_product(action)]))
        done=False
        next_state_int=agent.get_action_r(self._get_emb_product(action))
        
        if self._get_pr_from_index(next_state_int) in ('','[UNK]'):
            done=True
        next_state=self._get_pr_from_index(next_state_int)
        customer_interests,recommendation=self._get_cluster_from_cust_id(next_state,state)
        if recommendation[0] in customer_interests:
            reward=2
        reward=-1
        return next_state,reward,done
        
        

In [ ]:
env=recommendation_Env('../data/AMAZON_FASHION.csv',product_embeds,model1,prod_model)

In [ ]:
env.step(1245,agent,'[UNK]')

('b00wxmir4e', -1, False)

In [ ]:
# train(agent,env,2)
agent.get_action_r(model1(np.array([['7106116521']])))

54677

In [ ]:
# train(agent,env,1,model1)
import os
from tensorboard.plugins import projector

In [ ]:
# Set up a logs directory, so Tensorboard knows where to look for files.
log_dir='/logs/imdb-example/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for pid in Xtrain.pID.unique():
    f.write("{}\n".format(pid))
  # Fill in the rest of the labels with "unknown".
  for unknown in range(1, 186191):
    f.write("unknown #{}\n".format(unknown))


# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(model1.layers[2].get_weights()[0][1:])
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [ ]:
# Now run tensorboard against on log data we just saved.
%tensorboard --logdir /logs/imdb-example/

Reusing TensorBoard on port 6006 (pid 10288), started 0:09:01 ago. (Use '!kill 10288' to kill it.)

In [ ]:
a2c_model(np.array([['']]))

[<tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[-0.01369799]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1, 186191), dtype=float32, numpy=
 array([[[-0.00016137,  0.00032786, -0.00015888, ..., -0.00122273,
           0.00126742,  0.00157114]]], dtype=float32)>]

# its called embedding projector

In [ ]:
cust_emb_model(np.array([['A1D4G1SNUZWQOT']]))

<tf.Tensor: shape=(1, 1, 128), dtype=float32, numpy=
array([[[-4.7601487e-02, -3.5045672e-02, -2.0473052e-02,  7.7812187e-03,
          3.5366405e-02, -2.8126348e-02,  4.2544808e-02,  4.2236004e-02,
         -4.6327915e-02,  3.6636833e-02, -2.3304462e-02, -1.7544292e-02,
         -4.2753387e-02,  1.3804268e-02,  2.8625820e-02, -1.5286207e-03,
         -4.9141932e-02,  4.3871943e-02,  1.8376481e-02, -3.7240554e-02,
          2.2750173e-02,  8.7446086e-03,  4.9775355e-03,  1.1903822e-02,
         -4.1992329e-02,  3.4304857e-03, -9.2884786e-03,  1.9025330e-02,
          3.3402573e-02, -1.8713724e-02, -4.6140529e-02, -2.1862149e-02,
          3.5735536e-02, -1.5022110e-02, -3.9058030e-02, -4.7266509e-02,
         -3.7030578e-02,  1.9349862e-02,  4.4185985e-02, -1.2529992e-02,
          1.8964279e-02, -4.5682825e-02,  3.4722891e-02,  1.7310288e-02,
          3.0591860e-03,  4.1655753e-02, -3.7192296e-02, -3.9230660e-04,
         -4.3418493e-02, -1.1700392e-03, -4.5764830e-02,  2.5385011e-02

In [ ]:
cust_emb_model(np.array([['A1D4G1SNUZWQOT']]))

<tf.Tensor: shape=(1, 1, 128), dtype=float32, numpy=
array([[[-4.7601487e-02, -3.5045672e-02, -2.0473052e-02,  7.7812187e-03,
          3.5366405e-02, -2.8126348e-02,  4.2544808e-02,  4.2236004e-02,
         -4.6327915e-02,  3.6636833e-02, -2.3304462e-02, -1.7544292e-02,
         -4.2753387e-02,  1.3804268e-02,  2.8625820e-02, -1.5286207e-03,
         -4.9141932e-02,  4.3871943e-02,  1.8376481e-02, -3.7240554e-02,
          2.2750173e-02,  8.7446086e-03,  4.9775355e-03,  1.1903822e-02,
         -4.1992329e-02,  3.4304857e-03, -9.2884786e-03,  1.9025330e-02,
          3.3402573e-02, -1.8713724e-02, -4.6140529e-02, -2.1862149e-02,
          3.5735536e-02, -1.5022110e-02, -3.9058030e-02, -4.7266509e-02,
         -3.7030578e-02,  1.9349862e-02,  4.4185985e-02, -1.2529992e-02,
          1.8964279e-02, -4.5682825e-02,  3.4722891e-02,  1.7310288e-02,
          3.0591860e-03,  4.1655753e-02, -3.7192296e-02, -3.9230660e-04,
         -4.3418493e-02, -1.1700392e-03, -4.5764830e-02,  2.5385011e-02

In [ ]:
model1(np.array([['7106116521']]))

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[ 1.44032538e-02, -9.95387882e-02,  1.10612884e-02,
         6.79661054e-03, -4.09440510e-02,  1.00643009e-01,
        -1.96929991e-01, -3.08258086e-02,  3.37495469e-02,
         2.14395046e-01,  8.11212361e-02,  4.69950475e-02,
         2.28751451e-02, -2.81317890e-01, -1.18026860e-01,
         1.51767880e-01, -3.65776196e-02,  2.22208768e-01,
        -1.79001927e-01,  2.21132115e-01, -3.43354233e-02,
        -7.58359879e-02,  8.15490484e-02, -2.65507072e-01,
         8.10119137e-03, -1.77682005e-02,  9.54252258e-02,
         8.51073191e-02, -2.06482112e-01,  8.72338414e-02,
        -7.30938688e-02, -6.39760122e-02, -3.77323568e-01,
         2.23182198e-02,  1.82245344e-01,  1.60314262e-01,
        -3.36519064e-04, -2.05601931e-01,  1.85173184e-01,
         1.34607881e-01, -5.43930903e-02,  6.04644455e-02,
        -1.22840092e-01, -1.36357658e-02,  8.30601081e-02,
        -1.54992715e-01, -1.03358604e-01, -8.81951228e-02,
      